# October Code Jam

## Introduction 

This project explores sentiment analysis using Hugging Face Transformers, a leading library for Natural Language Processing (NLP). The goal is to automatically classify text as positive, negative, or neutral using pretrained transformer models.
 
We analyzed sentiment in social media posts, compared multiple models, and visualized their performance and confidence levels. We also applied the models to a creative dataset, showcasing how NLP can reveal insights from real-world text such as tweets, song lyrics, or news articles.


## Data Preprocessing

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
from transformers import pipeline
import matplotlib.pyplot as plt



In [ ]:
# Load dataset
